# 2. Using the ```plc.Planet``` class to fit light-curve

We start by creating a ```plc.Planet``` object for the planet of our interest:

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import pylightcurve as plc

planet = plc.get_planet('WASP-12b')

## 2.1 Adding light-curves to a ```plc.Planet``` object

Here we will crate a simulated light-curve, as we saw in the previous notebook:

In [ ]:
# define the filter
filter_name = 'COUSINS_R'

# define the out-of-transit observation time in hours
oot = 2

# define the exposure time in seconds
exp_time = 60

# define the noise (relatve flux)
noise = 0.005



# calculate the transit duration 
transit_duration = planet.transit_duration()

# define the time array
time_array = np.arange(planet.mid_time - 0.5 * transit_duration - oot/24, 
                       planet.mid_time + 0.5 * transit_duration + oot/24, exp_time/60/60/24)

# calculate model and add trend and noise
flux_array = planet.transit_integrated(time_array, exp_time=exp_time, filter_name=filter_name)
dt = (time_array - min(time_array))/(max(time_array)- min(time_array))
flux_array *= 4.3 * (1 + 0.001*dt - 0.01*dt*dt)
flux_array += np.random.normal(0, noise, len(flux_array))

# define the flux uncertainty array
flux_unc_array = np.ones_like(flux_array) * noise

# calculate phase
phase_array = planet.planet_phase(time_array)



plt.figure()
plt.errorbar(phase_array, flux_array, flux_unc_array, c='k', fmt='o', ms=2)
plt.ylabel('Flux (relative)')
plt.xlabel('Phase')


In case we added other observations to the ``plc.Planet`` object before and we don't want them, we can remove them asd follows:

In [ ]:
planet.clear_observations() 

To add an observation to a ```plc.Planet``` object we can type:



In [ ]:

planet.add_observation(
    time = time_array,          # the time vector of our observation
                                # np.array of float values 
    
    time_format = 'BJD_TDB',    # format in which our time vector is expressed
                                # str, available formats are: JD_UTC, MJD_UTC, HJD_UTC, HJD_TDB, BJD_UTC, BJD_TDB 

    exp_time = 60,              # exposure time of our time vector
                                # float, in seconds
        
    time_stamp = 'mid',         # exposure time stamp for our time vector (do the numbers refer to the exposure start, the mid-exposure, or the exposure end?)
                                # str, available stamps are: start, mid, end 
    
    flux = flux_array,          # the flux vector of our observation
                                # np.array of float values, 
    
    flux_unc = flux_unc_array,  # the flux-uncertainty vector of our observation
                                # np.array of float values, 
    
    flux_format = 'flux',       # format in which our flux and flux-uncertainty vectors are expressed
                                # str, available formats are: flux, mag
    
    filter_name = 'COUSINS_R',  # filter used for this observation 
                                # str, available filters are: all the default filters and those added manually by us

    observatory_latitude=None,  # gedodetic latitude of the the observatory, for ground-based observations
                                # float (positive is North), in degrees, default value is None
    
    observatory_longitude=None, # gedodetic longitude of the the observatory, for ground-based observations
                                # float (positive is East), in degrees, default value is None
    
    wlrange=None,               # for the cases where a specroscopic band-pass is needed, we can also specify 
                                # a part of the filter's bandpass to be used, by defining the wlrange parameter 
                                # (by default is None). This is a list with two elements, indicating the limits 
                                # (in Angstrom) within the filter bandpass that we want to use. The wavelength 
                                # range needs to be within the limits of the filter bandpass, otherwise the 
                                # calsulation will fail.
    
    stellar_model='Phoenix_2018', # We may want to use different stellar models to calculate the 
                                  # limb-darkening coefficients There is a range of stellar models availabe 
                                  # throught the ExoTETHyS package pachage. The default stellar model is 
                                  # the Phoenix_2018. However some stellar parameters and some wavelengths, 
                                  # are not covered by all models. For example, Phoenix_2018 are not availble 
                                  # for the irac4 filter, or for very cool stars (e.g. 3000 K). The other 
                                  # stellar models are: Atlas_2000, Phoenix_2012_13, Phoenix_drift_2012, 
                                  # Stagger_2015, Stagger_2018. Please consult the the ExoTETHyS package 
                                  # page if an error occurs.
    
    auxiliary_data={},          # Time-series in addition to time, flux, and fluc_unc, that we want to provide.
                                # Dictonary of labeled np.arrays of float values
                                # e.g. auxiliary_data={'xpix': np.array([...]), 'ypix': np.array([...])}
                                # To see an example, check the notebook: detrending_examples/6_hd189733b_sst.ipynb 
    
    detrending_series='time',   # Time-series used for de-trending.
                                # str or list of str values
                                # Available series are: time, airmass (if the observatory location is provided) 
                                # We can also use the time-series provided as auxiliary data 
                                
    detrending_order=2,         # Polynomial order of the detrending function.
                                # int or list of int values
                                # e.g if detrending_series='time' and detrending_order=2, the trend fuction will be: 1 + At + Bt^2
                                # e.g if detrending_series='airmass' and detrending_order=1, the trend fuction will be: 1 + AX
                                # e.g if detrending_series=['time', airmass'] and detrending_order=[2,1], the trend fuction will be: (1 + At + Bt^2)(1 + CX)

    trend_function=None,        # Custom trend function
    trend_parameters=None,      # Parameters of the custom trend function
                                # To see an example, check the notebook: detrending_examples/4_wasp39b_hst.ipynb
)



## 2.2 Runing and optimising the fitting process

To fit the light-curve we just added with all the pre-defined options, and save the final results and plots in a directory names ``"results"``, we can just type:

In [ ]:
fitting_results = planet.transit_fitting('results')

The above returns a dictionary with all the fitting results (we will explore this dictionary later). The full range of options for the ```planet.transit_fitting()``` function, can be seen below:

| kwarg                        | type       |default value     | decription                                |
|:-----------------------------|:----------:|:----------------:|:-----------------------------------------:|
| ``output_folder``            |``str``     | ``None``         | Path to save fillitng output (plots, pickle, etc).|
| ``walkers``                  |``int``     | ``None``         | Number of MCMC walkers (None will result in walkers three times the number of parameters).|
| ``walkers_spread``           |``float``   | ``0.01``         | How much to sread the wakers around the initial state. ``walkers_spread`` is relative to the prior size and the default value is 1%.|
| ``iterations``               |``int``     | ``None``         | Number of MCMC iterations (None will result in 5000 iterations).|
| ``burn_in``                  |``int``     | ``None``         | Number of initial MCMC iterations to be rejected (None will result in rejecting the first 1000 iterations)
| ``optimiser``                |``str``     | ``'emcee'``      | Optimizer to be used. We can change this to ``'curve_fit'`` for a quick test before runing the MCMC.
| ``fit_ldc1``                 |``bool``    | ``False``        | Let the first limb-darkening coefficient as a free parameter or not.|
| ``fit_ldc2``                 |``bool``    | ``False``        | Let the second limb-darkening coefficient as a free parameter or not.|
| ``fit_ldc3``                 |``bool``    | ``False``        | Let the third limb-darkening coefficient as a free parameter or not.|
| ``fit_ldc4``                 |``bool``    | ``False``        | Let the forth limb-darkening coefficient as a free parameter or not.|
| ``fit_individual_rp_over_rs``|``bool``    | ``False``        | Use a different Rp/Rs parameter for every filter or not.|
| ``fit_sma_over_rs``          |``bool``    | ``False``        | Let the a/Rs as a free parameter or not.|
| ``fit_inclination``          |``bool``    | ``False``        | Let the inclination as a free parameter or not.|
| ``fit_mid_time``             |``bool``    | ``True``         | Let the transit mid-time as a free parameter or not.|
| ``fit_period``               |``bool``    | ``False``        | Let the period as a free parameter or not.|
| ``fit_individual_times``     |``bool``    | ``True``         | Use a different mid-time parameter for every epoch or not. If ``False``, the parameter to be fitted is the ephemeris mid-time. We cannot set ``fit_individual_times=True`` and ``fit_period=True`` at the same time. |
| ``fit_ldc_limits``           |``list``    | ``[-3.0,3.0]``   | Additive fitting limits for the limb-darkening coefficients (no units). The dafault limits are between -3.0 and +3.0 from the initial value.|
| ``fit_rp_over_rs_limits``    |``list``    | ``[0.1,10.0]``   | Relative fitting limits for the Rp/Rs (no units). The dafault limits are between 10% and 10 times the initial value.|
| ``fit_sma_over_rs_limits``   |``list``    | ``[0.001,1000]`` | Relative fitting limits for the a/Rs (no units). The dafault limits are between 0.1% and 1000 times the initial value.|
| ``fit_inclination_limits``   |``list``    | ``[10.0,90.0]`` | Absolute fitting limits for the inclination (degrees). The dafault limits are between 10 degres and 90 degrees. |
| ``fit_mid_time_limits``      |``list``    | ``[-0.2,0.2]``   | Additive fitting limits for the mid-time (days). The dafault limits are between -0.2 days and +0.2 days from the initial value.|
| ``fit_period_limits``        |``list``    | ``[0.001,1000]`` | Relative fitting limits for the period (days). The dafault limits are between 0.1% and 1000 times the initial value.|
| ``optimise_initial_parameters_trials`` |``int``    | ``3``         | There is a first step of maximizing the probability function (and therefore optimisig the initial parameters) using the ``numpy.minimize`` function. This parameter sets the maximum trials. If the optimisation keeps failing we can set this to ``0``.|
| ``strech_prior``             |``float``    | ``0.2``         | How much to vary the parameters around the initial state during optimisation. ``strech_prior`` is relative to the prior size and the default value is 20%.|
| ``scale_uncertainties``      |``bool``    | ``False``        | Scale the provided uncertainties based on RMS of the residuats of the preliminary fit or not. If ``True`` the final reduced chi square is forced to be close to 1. |  
| ``filter_outliers``          |``bool``    | ``False``        | Filter the input data to remove 3-sigma outliers or not.|
| ``return_traces``            |``bool``    | ``True``         | Include the MCMC chains in the final dictionary or not.|
| ``counter``                  |``str``     | ``'Transit fitting'``| Name of the counter that apears while running the fitting.|

If the light-curve was an eclise light-curve, then we should use:
``` python
fitting_results = planet.eclipse_fitting()
```

The full range of options for the ```planet.eclipse_fitting()``` function, can be seen below:

| kwarg                        | type       |default value     | decription                                |
|:-----------------------------|:----------:|:----------------:|:-----------------------------------------:|
| ``output_folder``            |``str``     | ``None``         | Path to save fillitng output (plots, pickle, etc).|
| ``walkers``                  |``int``     | ``None``         | Number of MCMC walkers (None will result in walkers three times the number of parameters).|
| ``walkers_spread``           |``float``   | ``0.01``         | How much to sread the wakers around the initial state, relative to the prior size. Default is 1%.|
| ``iterations``               |``int``     | ``None``         | Number of MCMC iterations (None will result in 5000 iterations).|
| ``burn_in``                  |``int``     | ``None``         | Number of initial MCMC iterations to be rejected (None will result in rejecting the first 1000 iterations)
| ``optimiser``                |``str``     | ``'emcee'``      | Optimizer to be used. We can change this to ``'curve_fit'`` for a quick test before runing the MCMC.
| ``fit_individual_fp_over_rs``|``bool``    | ``True``         | Use a different Fp/Rs parameter for every filter or not.|
| ``fit_rp_over_rs``           |``bool``    | ``False``        | Let the Rp/Rs as a free parameter or not.|
| ``fit_individual_rp_over_rs``|``bool``    | ``False``        | Use a different Rp/Rs parameter for every filter or not.|
| ``fit_sma_over_rs``          |``bool``    | ``False``        | Let the a/Rs as a free parameter or not.|
| ``fit_inclination``          |``bool``    | ``False``        | Let the inclination as a free parameter or not.|
| ``fit_mid_time``             |``bool``    | ``True``         | Let the transit mid-time as a free parameter or not.|
| ``fit_period``               |``bool``    | ``False``        | Let the period as a free parameter or not.|
| ``fit_individual_times``     |``bool``    | ``True``         | Use a different mid-time parameter for every epoch or not. If ``False``, the parameter to be fitted is the ephemeris mid-time. We cannot set ``fit_individual_times=True`` and ``fit_period=True`` at the same time. |
| ``fit_fp_over_rs_limits``    |``list``    | ``[0.1,10.0]``   | Relative fitting limits for the Fp/Rs (no units). The dafault limits are between 10% and 10 times the initial value.|
| ``fit_rp_over_rs_limits``    |``list``    | ``[0.1,10.0]``   | Relative fitting limits for the Rp/Rs (no units). The dafault limits are between 10% and 10 times the initial value.|
| ``fit_sma_over_rs_limits``   |``list``    | ``[0.001,1000]`` | Relative fitting limits for the a/Rs (no units). The dafault limits are between 0.1% and 1000 times the initial value.|
| ``fit_inclination_limits``   |``list``    | ``[10.0,90.0]`` | Absolute fitting limits for the inclination (degrees). The dafault limits are between 10 degres and 90 degrees. |
| ``fit_mid_time_limits``      |``list``    | ``[-0.2,0.2]``   | Additive fitting limits for the mid-time (days). The dafault limits are between -0.2 days and +0.2 days from the initial value.|
| ``fit_period_limits``        |``list``    | ``[0.001,1000]`` | Relative fitting limits for the period (days). The dafault limits are between 0.1% and 1000 times the initial value.|
| ``optimise_initial_parameters_trials`` |``int``    | ``3``         | There is a first step of maximizing the probability function (and therefore optimisig the initial parameters) using the ``numpy.minimize`` function. This parameter sets the maximum trials. If the optimisation keeps failing we can set this to ``0``.|
| ``strech_prior``             |``float``    | ``0.2``         | How much to vary the parameters around the initial state during optimisation. ``strech_prior`` is relative to the prior size and the default value is 20%.|
| ``scale_uncertainties``      |``bool``    | ``False``        | Scale the provided uncertainties based on RMS of the residuats of the preliminary fit or not. If ``True`` the final reduced chi square is forced to be close to 1. |  
| ``filter_outliers``          |``bool``    | ``False``        | Filter the input data to remove 3-sigma outliers or not.|
| ``return_traces``            |``bool``    | ``True``         | Include the MCMC chains in the final dictionary or not.|
| ``counter``                  |``str``     | ``'Eclipse fitting'``| Name of the counter that apears while running the fitting.|

## 2.3 Checking the fitting results



By the term "global" we will refer to the combination all the observations addded to the ```plc.Planet``` object. More often, it is usful to investigations the results for individual observations (found in ```fitting_results['observations']```).

Lets check now what we can find under **``fitting_results``**:

* ``'settings'``: Includes all the fitting settings, as defined when running the ``planet.transit_fitting()`` function.
   - ``'output_folder'``
   - ``'iterations'``
   - ``'walkers'``
   - ``'burn_in'``
   - ``'strech_prior'``
   - ``'walkers_spread'``
   - ``'optimise_initial_parameters'``
   - ``'scale_uncertainties'``
   - ``'filter_outliers'``
   - ``'optimiser'``
   - ``'fit_ldc1'``
   - ``'fit_ldc2'``
   - ``'fit_ldc3'``
   - ``'fit_ldc4'``
   - ``'fit_individual_rp_over_rs'``
   - ``'fit_sma_over_rs'``
   - ``'fit_inclination'``
   - ``'fit_mid_time'``
   - ``'fit_period'``
   - ``'fit_individual_times'``
   - ``'fit_ldc_limits'``
   - ``'fit_rp_over_rs_limits'``
   - ``'fit_sma_over_rs_limits'``
   - ``'fit_inclination_limits'``
   - ``'fit_mid_time_limits'``
   - ``'fit_period_limits'``
* ``'input_series'``: Global input time-series.
    - ``'time'``: Global time array, in BJD_TDB.
    - ``'flux'``: Global flux array, in relative flux.
    - ``'flux_unc'``: Global flux uncertainty array, in relative flux. 
    - Plus any auxiliary series provided by the user.
* ``'parameters'``: Gloal model and trend parameters.

    - (example) ``'n'``:
        - ``'name'``: Name of the parameter.
        - ``'print_name'``: Name of the parameter as shown on the plots. 
        - ``'initial'``: Initial value of the parameter.
        - ``'min_allowed'``: Minimum allowed value of the parameter.
        - ``'max_allowed'``: Maximum allowed value of the parameter.
        - ``'trace'``: MCMC chain of the parameter.
        - ``'trace_bins'``: Bins of the 1D distribution of the MCMC chain of the parameter.
        - ``'trace_counts'``: Counts of the 1D distribution of the MCMC chain of the parameter.
        - ``'value'``: Median value of the MCMC chain of the parameter (final reported value).
        - ``'m_error'``: Median - 16th percentile of the MCMC chain of the parameter.
        - ``'p_error'``: 84th percentile - median of the MCMC chain of the parameter.
        - ``'print_value'``: Rounded value of the paraneter.
        - ``'print_m_error'``: Rounded lower uncertainty of the paraneter.
        - ``'print_p_error'``: Rounded upper uncertainty of the paraneter.
* ``'output_series'``: 
    - ``'model'``: Global model, in relative flux.
    - ``'residuals'``: Global residuals, in relative flux.
    - ``'trend'``: Global trend, in relative flux. When the model is devided by the trend we get the global de-trended model.
* ``'statistics'``: 
    - ``'corr_matrix'``: Correlation matrix of the traces of the free parameters.
    - ``'corr_variables'``: Names of the free parameters included in the correlation matrix.
    - ``'res_autocorr'``: Autocorrelation function of the normalised residuals.
    - ``'res_max_autocorr'``: Max. autocorrelation of the normalised residuals.
    - ``'res_max_autocorr_flag'``: Autocorrelation flag: Max. autocorrelation of the normalised residuals > Max. autocorrelation of white noise.
    - ``'res_shapiro'``: Shapiro test of the normalised residuals.
    - ``'res_shapiro_flag'``: Shapiro flag: Shapiro test of the normalised residuals > Shapiro test of white noise.
    - ``'res_mean'``: Mean value of the residuals.
    - ``'res_std'``: STD of the residuals.
    - ``'res_rms'``: RMS of the residuals.
    - ``'res_chi_sqr'``: Chi squared of the residuals.
    - ``'res_red_chi_sqr'``: Reduced chi squared of the residuals.
* ``'detrended_series'``: 
    - ``'time'``: Global time array, in BJD_TDB.
    - ``'flux'``: Global flux array devided by the trend model, in relative flux.
    - ``'flux_unc'``: Global flux uncertainty array devided by the trend model, in relative flux. 
    - ``'model'``: Global model devided by the trend model, in relative flux.
    - ``'residuals'``: Global residuals devided by the trend model, in relative flux.
* ``'detrended_statistics'``:
    - ``'res_autocorr'``: Autocorrelation function of the normalised detrended residuals.
    - ``'res_max_autocorr'``: Max. autocorrelation of the normalised detrended residuals.
    - ``'res_max_autocorr_flag'``: Autocorrelation flag: Max. autocorrelation of the normalised detrended residuals > Max. autocorrelation of white noise.
    - ``'res_shapiro'``: Shapiro test of the normalised detrended residuals.
    - ``'res_shapiro_flag'``: Shapiro flag: Shapiro test of the normalised detrended residuals > Shapiro test of white noise.
    - ``'res_mean'``: Mean value of the detrended residuals.
    - ``'res_std'``: STD of the detrended residuals.
    - ``'res_rms'``: RMS of the detrended residuals.
    - ``'res_chi_sqr'``: Chi squared of the detrended residuals.
    - ``'res_red_chi_sqr'``: Reduced chi squared of the detrended residuals.
* ``'observations'``: 
    - ``'obs0'``: Settings and results for the first indivudual light-curve.
    - ``'obs1'``: Settings and results for the second indivudual light-curve.
    - ``'obs2'``: Settings and results for the third indivudual light-curve.
    - etc...

Lets check now what we can find under **``fitting_results['observations']['obs0]``**:

- ``'original_data'``:  Includes all the input arguments provided by the user when the observation was added to the planet ``plc.Planet`` object.
    - ``'time'``
    - ``'time_format'`` 
    - ``'exp_time'`` 
    - ``'time_stamp'`` 
    - ``'flux'`` 
    - ``'flux_unc'`` 
    - ``'flux_format'``
    - ``'filter_name'``
    - ``'wlrange'`` 
    - ``'stellar_model'`` 
    - ``'observatory_longitude'`` 
    - ``'observatory_latitude'`` 
    - ``'auxiliary_data'``
    - ``'trend_function'``
    - ``'trend_parameters'``
    - ``'detrending_series'``
    - ``'detrending_order'``
- ``'model_info'``: Includes extra parameters adopted by the ``plc.Planet`` object and used for modeling
    - ``'target'``: Name of the planet.
    - ``'date'``: Date of observation.
    - ``'ldc_method'``
    - ``'max_sub_exp_time'``
    - ``'precision'``
    - ``'stellar_temperature'``
    - ``'stellar_logg'``
    - ``'stellar_metallicity'``
    - ``'limb_darkening_coefficients'``
    - ``'emissivity'``
    - ``'albedo'``
    - ``'sma_over_rs'``
    - ``'observation_type'``: Transit or eclipse.
    - ``'epoch'``: Epoch of transit or eclipse.
    - ``'obs_id'``: Observation ID (here ``'obs0'``)
- ``'data_conversion_info'``: 
    - ``'notes'``: Notes on data conversions that were implemented.
    - ``'ra'``: RA used to conver to BJD_TDB.
    - ``'dec'``: DEC used to conver to BJD_TDB.
    - ``'time_stamp_correction'``: Correction on the time array due to the exposure time-stamp.
    - ``'bjd_tdb_correction'``: BJD_TDB correction added to the time array.
    - ``'scale_factor'``: Scale factor applied to the initial fluc uncertainties.
    - ``'outliers'``: Number of outliers removed.
    - ``'non_outliers_map'``: Indices of the data point that were not removed as outliers.
* ``'input_series'``: Global input time-series.
    - ``'time'``: Global time array, in BJD_TDB.
    - ``'flux'``: Global flux array, in relative flux.
    - ``'flux_unc'``: Global flux uncertainty array, in relative flux. 
    - Plus any auxiliary series provided by the user.
* ``'parameters'``: Gloal model and trend parameters.

    - (example) ``'n'``:
        - ``'name'``: Name of the parameter.
        - ``'print_name'``: Name of the parameter as shown on the plots. 
        - ``'initial'``: Initial value of the parameter.
        - ``'min_allowed'``: Minimum allowed value of the parameter.
        - ``'max_allowed'``: Maximum allowed value of the parameter.
        - ``'trace'``: MCMC chain of the parameter.
        - ``'trace_bins'``: Bins of the 1D distribution of the MCMC chain of the parameter.
        - ``'trace_counts'``: Counts of the 1D distribution of the MCMC chain of the parameter.
        - ``'value'``: Median value of the MCMC chain of the parameter (final reported value).
        - ``'m_error'``: Median - 16th percentile of the MCMC chain of the parameter.
        - ``'p_error'``: 84th percentile - median of the MCMC chain of the parameter.
        - ``'print_value'``: Rounded value of the paraneter.
        - ``'print_m_error'``: Rounded lower uncertainty of the paraneter.
        - ``'print_p_error'``: Rounded upper uncertainty of the paraneter.
* ``'output_series'``: 
    - ``'model'``: Global model, in relative flux.
    - ``'residuals'``: Global residuals, in relative flux.
    - ``'trend'``: Global trend, in relative flux. When the model is devided by the trend we get the global de-trended model.
* ``'statistics'``: 
    - ``'res_autocorr'``: Autocorrelation function of the normalised residuals.
    - ``'res_max_autocorr'``: Max. autocorrelation of the normalised residuals.
    - ``'res_max_autocorr_flag'``: Autocorrelation flag: Max. autocorrelation of the normalised residuals > Max. autocorrelation of white noise.
    - ``'res_shapiro'``: Shapiro test of the normalised residuals.
    - ``'res_shapiro_flag'``: Shapiro flag: Shapiro test of the normalised residuals > Shapiro test of white noise.
    - ``'res_mean'``: Mean value of the residuals.
    - ``'res_std'``: STD of the residuals.
    - ``'res_rms'``: RMS of the residuals.
    - ``'res_chi_sqr'``: Chi squared of the residuals.
    - ``'res_red_chi_sqr'``: Reduced chi squared of the residuals.
* ``'detrended_series'``: 
    - ``'time'``: Global time array, in BJD_TDB.
    - ``'flux'``: Global flux array devided by the trend model, in relative flux.
    - ``'flux_unc'``: Global flux uncertainty array devided by the trend model, in relative flux. 
    - ``'model'``: Global model devided by the trend model, in relative flux.
    - ``'residuals'``: Global residuals devided by the trend model, in relative flux.
* ``'detrended_statistics'``:
    - ``'res_autocorr'``: Autocorrelation function of the normalised detrended residuals.
    - ``'res_max_autocorr'``: Max. autocorrelation of the normalised detrended residuals.
    - ``'res_max_autocorr_flag'``: Autocorrelation flag: Max. autocorrelation of the normalised detrended residuals > Max. autocorrelation of white noise.
    - ``'res_shapiro'``: Shapiro test of the normalised detrended residuals.
    - ``'res_shapiro_flag'``: Shapiro flag: Shapiro test of the normalised detrended residuals > Shapiro test of white noise.
    - ``'res_mean'``: Mean value of the detrended residuals.
    - ``'res_std'``: STD of the detrended residuals.
    - ``'res_rms'``: RMS of the detrended residuals.
    - ``'res_chi_sqr'``: Chi squared of the detrended residuals.
    - ``'res_red_chi_sqr'``: Reduced chi squared of the detrended residuals.

We easily check the results for individual parameters as follows:

In [ ]:
rp_over_rs = fitting_results['observations']['obs0']['parameters']['rp_over_rs']['value']
rp_over_rs_e1 = fitting_results['observations']['obs0']['parameters']['rp_over_rs']['m_error']
rp_over_rs_e2 = fitting_results['observations']['obs0']['parameters']['rp_over_rs']['p_error']

print('rp_over_rs = {0}(-{1}, +{2})'.format(rp_over_rs, rp_over_rs_e1, rp_over_rs_e2))

mid_time = fitting_results['observations']['obs0']['parameters']['mid_time']['value']
mid_time_e1 = fitting_results['observations']['obs0']['parameters']['mid_time']['m_error']
mid_time_e2 = fitting_results['observations']['obs0']['parameters']['mid_time']['p_error']

print('mid_time = {0}(-{1}, +{2})'.format(mid_time, mid_time_e1, mid_time_e2))

We can plot the raw light-cure the trend and the residuals as follows:

In [ ]:
observation = fitting_results['observations']['obs0']
plt.figure(figsize=(9,6))
plt.subplot(2, 1, 1)
plt.errorbar(observation['input_series']['time'], 
             observation['input_series']['flux'], 
             observation['input_series']['flux_unc'], 
             fmt='o', zorder=0, label='raw data')
plt.plot(observation['input_series']['time'], observation['output_series']['trend'], 'r--', zorder=1, 
         label='trend model')
plt.plot(observation['input_series']['time'], observation['output_series']['model'], 'r-', zorder=1,
         label='trend + transit model'
        )
plt.legend(fontsize='small')
plt.ylabel('rel. flux')

plt.subplot(2, 1, 2)
plt.errorbar(observation['input_series']['time'], 
             observation['output_series']['residuals'], 
             observation['input_series']['flux_unc'], 
             fmt='o', zorder=0)
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.ylabel('rel. flux residuals')


We can also plot the detrended light-cure the residuals as follows:

In [ ]:
observation = fitting_results['observations']['obs0']
plt.figure(figsize=(9,6))
plt.subplot(2, 1, 1)
plt.errorbar(observation['detrended_series']['time'], 
             observation['detrended_series']['flux'], 
             observation['detrended_series']['flux_unc'], 
             fmt='o', zorder=0, label='detrended data')
plt.plot(observation['detrended_series']['time'], observation['detrended_series']['model'], 'r-', zorder=1,
         label='transit model'
        )
plt.legend(fontsize='small')
plt.ylabel('rel. flux')

plt.subplot(2, 1, 2)
plt.errorbar(observation['detrended_series']['time'], 
             observation['detrended_series']['residuals'], 
             observation['detrended_series']['flux_unc'], 
             fmt='o', zorder=0)
plt.xlabel(r'Time (BJD$_\mathrm{TDB}$)')
plt.ylabel('rel. flux residuals')
